In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
encoder = OneHotEncoder(sparse=False)
time_encoded = encoder.fit_transform(train_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
train_data = pd.concat([train_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['ID', '풍속 (m/s)'], axis=1)  # 입력 변수들
y_train = train_data['풍속 (m/s)']  # 출력 변수 (풍속)

# 랜덤 포레스트 회귀 모델과 엑스트라 트리 회귀 모델, 그리고 신경망 회귀 모델을 사용하여 풍속 예측 모델을 학습시킵니다.
rf_model = RandomForestRegressor(random_state=42)
et_model = ExtraTreesRegressor(random_state=42)
nn_model = MLPRegressor(random_state=42, max_iter=1000)
rf_model.fit(X_train, y_train)
et_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')

# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
time_encoded = encoder.transform(test_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
test_data = pd.concat([test_data, time_encoded_df], axis=1).drop(['측정 시간대'], axis=1)

# 테스트 데이터로 풍속 예측을 수행합니다.
X_test = test_data.drop('ID', axis=1)  # 테스트 입력 변수들
y_pred_rf = rf_model.predict(X_test)  # 랜덤 포레스트로 풍속 예측
y_pred_et = et_model.predict(X_test)  # 엑스트라 트리로 풍속 예측
y_pred_nn = nn_model.predict(X_test)  # 신경망으로 풍속 예측

# 세 모델의 예측값을 평균하여 최종 예측값으로 사용합니다.
y_pred_ensemble = (y_pred_rf + y_pred_et + y_pred_nn) / 3

# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')
submission['풍속 (m/s)'] = y_pred_ensemble

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")

# Cross Validation을 활용하여 평균 RMSE 계산
rf_scores = cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error', cv=KFold(n_splits=5, shuffle=True, random_state=42))
et_scores = cross_val_score(et_model, X_train, y_train, scoring='neg_mean_squared_error', cv=KFold(n_splits=5, shuffle=True, random_state=42))
nn_scores = cross_val_score(nn_model, X_train, y_train, scoring='neg_mean_squared_error', cv=KFold(n_splits=5, shuffle=True, random_state=42))
ensemble_pred = (rf_model.predict(X_train) + et_model.predict(X_train) + nn_model.predict(X_train)) / 3
ensemble_scores = cross_val_score(ExtraTreesRegressor(), X_train, y_train - ensemble_pred, scoring='neg_mean_squared_error', cv=KFold(n_splits=5, shuffle=True, random_state=42))

# 각 모델의 평균 RMSE 출력
rf_rmse = np.sqrt(-rf_scores).mean()
et_rmse = np.sqrt(-et_scores).mean()
nn_rmse = np.sqrt(-nn_scores).mean()
ensemble_rmse = np.sqrt(-ensemble_scores).mean()

print("랜덤 포레스트 평균 RMSE:", rf_rmse)
print("엑스트라 트리 평균 RMSE:", et_rmse)
print("신경망 평균 RMSE:", nn_rmse)
print("앙상블 평균 RMSE:", ensemble_rmse)


C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  del sys.path[0]
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function g

풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.
랜덤 포레스트 평균 RMSE: 0.6083032518940009
엑스트라 트리 평균 RMSE: 0.5390010149541583
신경망 평균 RMSE: 1.4010978528707196
앙상블 평균 RMSE: 0.24824263483263803
